In [5]:
import glob
import pandas as pd
import numpy as np
import scipy
import matlab.engine
from sklearn.preprocessing import minmax_scale
mat = matlab.engine.start_matlab()

In [2]:
#信号を正規化する関数
def normalization(data_array):
    normalizated_data = minmax_scale(data_array)
    return normalizated_data

#信号を実効値信号に変換する関数(shakeelが使用)
def rms(data_array):
    square_data_array = data_array**2
    abs_square_data_array = np.abs(square_data_array)
    sum_abs_square_data = np.sum(abs_square_data_array)
    sum_mean_square_data = sum_abs_square_data / len(data_array)
    rms_data = np.sqrt(data_array / sum_mean_square_data)
    #standardizated_data = data_array / rms_data
    return rms_data

In [10]:
days = "01-29"
merged_data_path = glob.glob('csv/merged/merged_2023-'+ days + '*')[0]
merged_data_df = pd.read_csv(merged_data_path,names = ["Time","V(m/s)","T(C)","H(%RH)","φ(mG)"],skiprows=1)

#info
gauss_df = merged_data_df['φ(mG)']
wind_vel_df = merged_data_df['V(m/s)']

gauss_array = np.array(gauss_df.values[1:], dtype = 'float')
wind_vel_array = np.array(wind_vel_df.values[1:], dtype = 'float')

standardizated_gauss_array = normalization(gauss_array)
standardizated_gauss_array  = standardizated_gauss_array[:29000]
standardizated_wind_vel_array = normalization(wind_vel_array)
standardizated_wind_vel_array  = standardizated_wind_vel_array[:29000]


In [11]:
mat.cd(r'../matlab', nargout=0)
mel_spect_gauss_data = mat.mel_spectrogram_bad(matlab.double(standardizated_gauss_array),matlab.double(2900))
mel_spect_wind_vel_data = mat.mel_spectrogram_bad(matlab.double(standardizated_wind_vel_array),matlab.double(2900))
scipy.io.savemat('mat/gauss_mat/gauss-2023-'+ days + '-17.mat', {'gauss_feat':mel_spect_gauss_data})
scipy.io.savemat('mat/wind_mat/wind-2023-'+ days + '-17.mat', {'wind_vel_feat':mel_spect_wind_vel_data})